# Purpose:



## Imports:

In [1]:
# %matplotlib inline
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import ggplot as gp

# import mpld3
# # mpld3.enable_notebook()

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 60)
# import tables as h5

import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import statsmodels.stats.multitest as smm
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.formula import api as smform

import munch

import pybedtools as pbt

import pymc as mc

from shapely.geometry import LineString


import tabulate
tbl = tabulate.tabulate


from spartan.utils.genome_specific.GfusI1 import GfusI1_0
from spartan.utils.fastas import ParseFastA

from spartan.utils.annotations.ensembl.gff3 import parse_gff3
from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes
from spartan.utils.files import tableFile2namedTuple

from gs_ddRAD2015.scripts import ld_figures as ldfigs


/home/gus/anaconda2/envs/gs_2015/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/gus/anaconda2/envs/gs_2015/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
# set figure characteristics

# size
sns.set_context("poster", font_scale=1.0)

# Grid style
sns.set_style("whitegrid")

# colors
sns.set_palette(sns.hls_palette(n_colors=5, h=0.59, l=0.4, s=0.75))

## File paths:

In [4]:
# define paths to files
# out_dir = "/home/gus/data/gs_2015_ld/OT"
# out_dir = "/home/gus/data/gs_2015_ld/MS"
# out_dir = "/home/gus/data/gs_2015_ld/NB"
## out_dir = "/home/gus/data/gs_2015_ld/KG"
out_dir = "/home/gus/data/gs_2015_ld/OT_MS_NB_indiv.geno.ld/"

## Functional annatation (Argot2)
fanno = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

# gene models
btools_gene_models_gff3 = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/Glossina-fuscipes-IAEA_BASEFEATURES_GfusI1.1.gff3"

# Andrea's Genes


## Loading files

In [5]:
# gene_mods = pbt.BedTool(btools_gene_models_gff3).filter(lambda x: x[2] == 'gene').sort()#".saveas()

In [6]:
figs = ldfigs.Figures(out_dir=out_dir, formats='png')

In [7]:
figs.load_data_tables()


ld_figures: loading data_tables.




In [8]:
tables = munch.Munch(figs.d)
tables.keys()

['ld',
 'd_bins',
 'contig_info',
 'd_bin_v_others_melt',
 'len_contigs_per_bin',
 'd_bin_v_others',
 'contigs_per_bin',
 'mean_bin_r2_all',
 'sp_contigs',
 'ld_contig']

In [9]:
ld = tables.ld
ld.head()

,Unnamed: 0,Unnamed: 0.1,CHR_A,BP_A,BP_B,N_INDV,R2,BP_DELTA,distance_bin,R2_scaled_for_B,alpha_param,beta_param,cdf,one_minus_cdf,one_minus_cdf_BH,MAP_succeeded
0,0,0,Scaffold0,13388,23408,48,1.000000,10020,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False
1,1,1,Scaffold0,13388,23418,48,1.000000,10030,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False
2,2,2,Scaffold0,13388,23421,48,1.000000,10033,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False
3,3,3,Scaffold0,13388,23431,48,0.975315,10043,10000,0.974840,NaN,NaN,0.999662,3.384154e-04,NaN,False
4,4,4,Scaffold0,13388,30544,49,0.366428,17156,17150,0.366562,NaN,NaN,0.862496,1.375042e-01,NaN,False


In [10]:
ld_contig = tables.ld_contig
ld_contig.head()

,Unnamed: 0,Unnamed: 0.1,CHR_A,BP_A,BP_B,N_INDV,R2,BP_DELTA,distance_bin,R2_scaled_for_B,alpha_param,beta_param,cdf,one_minus_cdf,one_minus_cdf_BH,MAP_succeeded,kk_name,scaf_name,length
0,0,0,Scaffold0,13388,23408,48,1.000000,10020,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False,KK351785.1,Scaffold0,3329503
1,1,1,Scaffold0,13388,23418,48,1.000000,10030,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False,KK351785.1,Scaffold0,3329503
2,2,2,Scaffold0,13388,23421,48,1.000000,10033,10000,0.999500,NaN,NaN,1.000000,2.605385e-07,NaN,False,KK351785.1,Scaffold0,3329503
3,3,3,Scaffold0,13388,23431,48,0.975315,10043,10000,0.974840,NaN,NaN,0.999662,3.384154e-04,NaN,False,KK351785.1,Scaffold0,3329503
4,4,4,Scaffold0,13388,30544,49,0.366428,17156,17150,0.366562,NaN,NaN,0.862496,1.375042e-01,NaN,False,KK351785.1,Scaffold0,3329503


In [11]:
def get_halfing_coords(xs, ys, num, tolerance):
    xs_return = []
    ys_return = []
    
    halfing_values = get_n_halfing_values(val=ys[0],n=num)[1:]
    
    for value in halfing_values:
        indexes_close_to_yi = get_indexes_close_to_yi(ys=ys, value=value, tolerance=tolerance)
        middle_item = get_first_item(indexes=indexes_close_to_yi)
        if middle_item is None:
            print 'None'
        else:
            # print indexes_close_to_yi
            # print xs[middle_item]
            # print ys[middle_item]
            xs_return.append(xs[middle_item])
            ys_return.append(ys[middle_item])
    
    return xs_return,ys_return
    
def get_indexes_close_to_yi(ys, value, tolerance):
    ilist = []
    for i,y in enumerate(ys):
        if abs(value-y) <= tolerance:
            ilist.append(i)
    return ilist
    
def get_n_halfing_values(val, n):
    rvalues = []
    c_val = val
    for i in range(n+1):
        c_val = c_val/2.0
        rvalues.append(c_val)
    return rvalues
        
def get_middle_item(indexes):
    if len(indexes) == 0:
        return None
    return indexes[int(len(indexes)/2)]

def get_first_item(indexes):
    if len(indexes) == 0:
        return None
    return indexes[0]

# Distance bt snp-pair vs R^2 all:

### Pivot for distance_bin stats

In [12]:
def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [13]:
ld_pv_distbin_by_mean_R2_cdfs = pd.pivot_table(data=ld, 
                                               values=["R2","one_minus_cdf_BH","one_minus_cdf"], 
                                               index="distance_bin", 
                                               columns=None, 
                                               aggfunc=np.mean, 
                                               fill_value=None, margins=False, dropna=True).reset_index()

ld_pv_distbin_by_mean_R2_cdfs.head()

,distance_bin,R2,one_minus_cdf,one_minus_cdf_BH
0,0,0.401710,0.540232,NaN
1,50,0.293258,0.549390,NaN
2,100,0.255214,0.541580,NaN
3,150,0.250560,0.538777,NaN
4,200,0.230920,0.547183,NaN


In [14]:
ld_pv_count_distbin = pd.pivot_table(data=ld,
                                     values="R2", 
                                     index="distance_bin", 
                                     columns=None, 
                                     aggfunc=len, 
                                     fill_value=None, margins=False, dropna=True).reset_index()


ld_pv_count_distbin.head()

,distance_bin,R2
0,0,36566
1,50,4274
2,100,2010
3,150,1204
4,200,1159


In [15]:
ld_pv_distbin_by_mean_R2_cdfs = ld_pv_distbin_by_mean_R2_cdfs.merge(right=ld_pv_count_distbin, 
                                    how='inner', on='distance_bin', 
                                    left_on=None, right_on=None,
                                    suffixes=('', '_count'),
                                    copy=True)
ld_pv_distbin_by_mean_R2_cdfs.head()
new_column_names = ld_pv_distbin_by_mean_R2_cdfs.columns.values
new_column_names[-1] = 'SNP-pairs'
ld_pv_distbin_by_mean_R2_cdfs.columns = new_column_names
ld_pv_distbin_by_mean_R2_cdfs.head()

,distance_bin,R2,one_minus_cdf,one_minus_cdf_BH,SNP-pairs
0,0,0.401710,0.540232,NaN,36566
1,50,0.293258,0.549390,NaN,4274
2,100,0.255214,0.541580,NaN,2010
3,150,0.250560,0.538777,NaN,1204
4,200,0.230920,0.547183,NaN,1159


In [16]:
ld_pv_distbin_by_mean_R2_cdfs_melt = pd.melt(ld_pv_distbin_by_mean_R2_cdfs, id_vars=['distance_bin'])
ld_pv_distbin_by_mean_R2_cdfs_melt.tail()

,distance_bin,variable,value
209979,3274650,SNP-pairs,1
209980,3274800,SNP-pairs,1
209981,3274850,SNP-pairs,1
209982,3275650,SNP-pairs,1
209983,3277750,SNP-pairs,2


In [17]:
d_bin_v_others_melt = figs.d.d_bin_v_others_melt

In [18]:
# mpld3.enable_notebook()
# mpld3.disable_notebook()

In [19]:
# pal = sns.choose_colorbrewer_palette("s")

# pal = sns.husl_palette(n_colors=4, h=0.6, s=0.9, l=0.65)
# pal = sns.hls_palette(n_colors=5, h=0.59, l=0.4, s=0.75)
# sns.palplot(pal)

# pal = sns.choose_cubehelix_palette()

In [20]:
# # sns.set_palette(pal)

# melted_r2 = ld_pv_distbin_by_mean_R2_cdfs_melt.query("variable == 'R2'").query("0 <= distance_bin <= 500000")

# d = melted_r2.distance_bin

# r2 = melted_r2.value
# r2_95centile = r2.quantile(.95)

# rgplt = sns.regplot(d, r2, data=None, x_estimator=None, x_bins=None, x_ci=95, 
#             scatter=True, fit_reg=False, ci=95, n_boot=10000, 
#             )

# # plt.plot(d,
# #          movingaverage(interval=r2, window_size=6),
# #          color='coral', alpha=0.3);

# # lowess regression
# delta = 0.01 * len(d)
# r2_lowess = lowess(r2,d,frac=0.02, it=0, delta=delta)
# r2_half = r2.max()/2.


# centile_line = [[0,r2_95centile],[r2_lowess[-1][0],r2_95centile]]
# halfing_line = [[0,r2_half],[r2_lowess[-1][0],r2_half]]

# halfing_geo = LineString(halfing_line)
# centile_geo = LineString(centile_line)
# lowess_geo = LineString(r2_lowess)

# isect_c = centile_geo.intersection(lowess_geo)
# isect_h = halfing_geo.intersection(lowess_geo)

# isect_c_x,isect_c_y = int(round(isect_c.x)), isect_c.y
# isect_h_x,isect_h_y = int(round(isect_h.x)), isect_h.y

# plt.plot(d,
#          r2_lowess[:,1],
#          alpha=1,
#          label="Lowess regression line");

# plt.plot([x[0] for x in halfing_line],
#          [y[1] for y in halfing_line],  
#          label=u"Half of maximum LD ($r^2$ = {r2_half:.4g}, {isect} bp)".format(isect=isect_h_x,r2_half=r2_half),
#          alpha=1
#         );

# # plt.plot([x[0] for x in centile_line],
# #          [y[1] for y in centile_line], 
# #          label="95th percentile of mean bin-wise LD ({isect} bp)".format(isect=isect_c_x),
# #          alpha=1
# #         );




# # plt.plot(0,0,label="test3",alpha=0)

# # plt.scatter(x=xs_hlf, y=ys_hlf, 
# #             s=200, c=u'k', marker=u'x', 
# #             alpha=None, linewidths=2);

# plt.xlabel(r"Distance between SNP-pairs (bp)");
# plt.ylabel(r"Mean bin-wise LD ($r^2$)");



# plt.legend()

# # print "xs_hlf, ys_hlf: ",xs_hlf, ys_hlf 



In [21]:
# sns.set_palette(pal)
SITE=out_dir.split('/')[-1]

sns.set_context("poster", font_scale=1.0)

scale = 1
dpi = 300

# plt.figure(figsize=(8*scale, 6*scale),dpi=dpi)

filtered_by_bin = ld_pv_distbin_by_mean_R2_cdfs.query("0 <= distance_bin <= 500000")

snppairs = filtered_by_bin["SNP-pairs"]
d = filtered_by_bin.distance_bin
r2 = filtered_by_bin.R2
r2_95centile = r2.quantile(.95)

scat = plt.scatter(x=d, y=r2, s=20, c=np.log2(snppairs), 
                   marker=u'o', cmap="cubehelix_r", 
                   norm=None, vmin=None, vmax=None, 
                   alpha=1, linewidths=0, verts=None, hold=None,label=None);
plt.colorbar(scat, label=r"log$_{2}$(pairs per bin)");


# lowess regression
delta = 0.01 * len(d)
r2_lowess = lowess(r2,d,frac=0.02, it=0, delta=delta)
r2_half = r2.max()/2.


point_2_line = [[0,0.2],[r2_lowess[-1][0],0.2]]
halfing_line = [[0,r2_half],[r2_lowess[-1][0],r2_half]]

halfing_geo = LineString(halfing_line)
point_2_geo = LineString(point_2_line)
lowess_geo = LineString(r2_lowess)

isect_2 = point_2_geo.intersection(lowess_geo)
isect_h = halfing_geo.intersection(lowess_geo)

isect_2_x,isect_2_y = int(round(isect_2.x)), isect_2.y
isect_h_x,isect_h_y = int(round(isect_h.x)), isect_h.y

plt.plot(d,
         r2_lowess[:,1],
         alpha=1,
         label="Lowess regression line");

plt.plot([x[0] for x in halfing_line],
         [y[1] for y in halfing_line],  
         label=u"Half of maximum LD ($r^2$ = {r2_half:.4g}, {isect} bp)".format(isect=isect_h_x,r2_half=r2_half),
         alpha=1
        );

# plt.plot([x[0] for x in point_2_line],
#          [y[1] for y in point_2_line], 
#          label="Crosses $r^2 = 0.2$ ({isect} bp)".format(isect=isect_2_x),
#          alpha=1
#         );




# plt.plot(0,0,label="test3",alpha=0)

# plt.scatter(x=xs_hlf, y=ys_hlf, 
#             s=200, c=u'k', marker=u'x', 
#             alpha=None, linewidths=2);

plt.xlabel(r"Distance between SNP-pairs (bp)");
plt.ylabel(r"Mean bin-wise LD ($r^2$)");
plt.title("Site {site}".format(site=SITE))


plt.legend();

# print "xs_hlf, ys_hlf: ",xs_hlf, ys_hlf 



In [22]:
r2.max()

0.40170997414554999

In [23]:
r2_half

0.20085498707277499

In [24]:
r2_95centile

0.12283816443061293

In [25]:
melted_r2.head()

NameError: name 'melted_r2' is not defined

In [ ]:
# sns.set_palette(pal)

melted_one_minus_cdf = ld_pv_distbin_by_mean_R2_cdfs_melt.query("variable == 'one_minus_cdf'").query("0 <= distance_bin <= 500000")

d = melted_one_minus_cdf.distance_bin

one_minus_cdf = melted_one_minus_cdf.value

rgplt = sns.regplot(d, one_minus_cdf, data=None, x_estimator=None, x_bins=None, x_ci=95, 
            scatter=True, fit_reg=True, ci=95, n_boot=10000)

# plt.plot(d,
#          movingaverage(interval=one_minus_cdf, window_size=6),
#          color='coral', alpha=0.3);

# lowess regression
delta = 0.01 * len(d)
one_minus_cdf_lowess = lowess(one_minus_cdf,d,frac=0.2, it=0, delta=delta)

plt.plot(d,
         one_minus_cdf_lowess[:,1],
         alpha=1,
         label="Lowess regression line");



plt.xlabel(r"Distance between SNP-pairs (bp)");
plt.ylabel(r"Mean bin-wise 1-CDF");



plt.legend()

# print "xs_hlf, ys_hlf: ",xs_hlf, ys_hlf 



-------------------